## Import Libraries and Load Data

In [1]:
# Import libraries

import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.decomposition import TruncatedSVD
import patsy

# Set seed
np.random.seed(4031)

In [2]:
# Data file locations and names

project_root_dir = "Data"
project_subdir_prefix = "fold_"
train_data_filename = "train.csv"
test_data_filename = "test.csv"


# The number of train/test data folders and the target RMSE for each
# train/test split in each folder

n_datasets = 10

In [3]:
# Get list of data subfolders, each with a separate training and test set.
# fold1 - fold5 have target RMSE 0.125, and fold6 - fold10 have target RMSE 0.135.

os_walk = os.walk(project_root_dir)
data_subdir_list = [subdirs for root, subdirs, files in os_walk][0]
n_subdirs = len(data_subdir_list)

assert(n_subdirs == n_datasets)

In [4]:
# Lists for training and test datasets

train_datasets = []
test_datasets = []


# Loop over subfolders and read in training/test datasets and test weekly sales.
# Use a loop instead of using os.walk directly to avoid "fold10" immediately following "fold1".

for subdir_num in np.arange(n_subdirs) + 1:
    subdir_num_str = str(subdir_num)
    train_datasets.append(pd.read_csv(os.path.join(project_root_dir,
                                                   project_subdir_prefix + subdir_num_str,
                                                   train_data_filename)))
    test_datasets.append(pd.read_csv(os.path.join(project_root_dir,
                                                   project_subdir_prefix + subdir_num_str,
                                                   test_data_filename)))

## Define Scoring function

In [5]:
# Define a WMAE function for scoring

def wmae():
    file_path = 'Data/test_with_label.csv'
    test = pd.read_csv(file_path)
    num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

## Preprocess Data

In [294]:
#Loop through data and pull apart date into week and year
def preprocess(data):
    #Split date into useful features
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 

    #One hot encode Wk
    data = pd.get_dummies(data, columns=['Wk'], prefix='Week')

    return data

## SVD Implementation (not working)

In [ ]:
for j in range(n_datasets):
    train = train_datasets[j]
    test = test_datasets[j]

    # Initialize the DataFrame to store predictions
    test_pred = pd.DataFrame()

    fold_train = preprocess(train)
    fold_test = preprocess(test)

    stores = fold_train["Store"].unique()
    depts = fold_train["Dept"].unique()
    years = fold_train["Yr"].unique()

    for store in stores:
        for dept in depts:
            for year in years:
                #Find training and test data within same store and then same department
                train = fold_train[(fold_train["Store"] == store) & (fold_train["Dept"] == dept) & (fold_train["Yr"] == year)]
                test = fold_test[(fold_test["Store"] == store) & (fold_test["Dept"] == dept) & (fold_test["Yr"] == year)]

                #abort if the train data is non-existant (i.e., this combo of store/dept doesnt appear in data)
                if len(train) == 0:
                    continue

                Y_train = train["Weekly_Sales"]
                X_train = train.drop(["Weekly_Sales", "Date"], axis=1)

                X_test = test

                #Keep Store, dept, and date info for later merging
                tmp_pred = X_test[['Store', 'Dept', 'Date']]
                X_test = X_test.drop(["Date"], axis=1)


                #Implement SVD
                #n_components=5
                #svd_df = pd.DataFrame(svd_result, columns=[[f'SVD_{i}' for i in range (n_components)]]) 
                svd = TruncatedSVD() 
                svd_result = svd.fit_transform(X_train)
                svd_df = pd.DataFrame(svd_result) 

                #Train model on only the features SVD selected
                model = sm.OLS(Y_train, svd_df).fit()
                mycoef = model.params.fillna(0)
                
                #Fit SVD columns for test set
                X_test = svd.transform(X_test)
                
                #Predict Y
                tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)

                #Readd context of store, dept, and date
                test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
            
        
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    # Save the output to CSV
    file_path = f'Data/fold_{j+1}/mypred.csv'
    print(f'fold_{j+1} processed')
    test_pred.to_csv(file_path, index=False)


## Linear Refactor (not working)

In [ ]:
for j in range(n_datasets):
    train = train_datasets[j]
    test = test_datasets[j]

    # Initialize the DataFrame to store predictions
    test_pred = pd.DataFrame()

    fold_train = preprocess(train)
    fold_test = preprocess(test)

    stores = fold_train["Store"].unique()
    depts = fold_train["Dept"].unique()
    years = fold_train["Yr"].unique()

    for store in stores:
        for dept in depts:
            for year in years:
                #Find training and test data within same store and then same department
                train = fold_train[(fold_train["Store"] == store) & (fold_train["Dept"] == dept) & (fold_train["Yr"] == year)]
                test = fold_test[(fold_test["Store"] == store) & (fold_test["Dept"] == dept) & (fold_test["Yr"] == year)]

                #abort if the train data is non-existant (i.e., this combo of store/dept doesnt appear in data)
                if len(train) == 0:
                    continue

                Y_train = train["Weekly_Sales"]
                X_train = train.drop(["Weekly_Sales", "Date"], axis=1)

                X_test = test

                #Keep Store, dept, and date info for later merging
                tmp_pred = X_test[['Store', 'Dept', 'Date']]
                X_test = X_test.drop(["Date"], axis=1)

                X_train = X_train.drop(["Store", "Dept", "IsHoliday"], axis=1)
                X_test = X_test.drop(["Store", "Dept", "IsHoliday"], axis=1)

                #Add intercept columns
                X_train["Intercept"] = 1
                X_test["Intercept"] = 1

                #Cast one hot bools to ints
                X_train = X_train.astype(int)
                X_test = X_test.astype(int)

                #Train model on only the features SVD selected
                model = sm.OLS(Y_train, X_train).fit()
                mycoef = model.params.fillna(0)
                
                #Predict Y
                tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)

                #Readd context of store, dept, and date
                test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
            
        
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    # Save the output to CSV
    file_path = f'Data/fold_{j+1}/mypred.csv'
    print(f'fold_{j+1} processed')
    test_pred.to_csv(file_path, index=False)

## Original Code (OLS only)

In [6]:
#OLS method requires a different preprocess function

def preprocess(data):
    #Split date into useful features
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 

    return data


In [ ]:
#Original OLS model

# Loop over folds
for j in range(n_datasets):
    
    # Get a pair of training and test sets
    train = train_datasets[j]
    test = test_datasets[j]

    test_pred = pd.DataFrame()

    # Identify the distinct store/dept pairs shared by the training and test set.
    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])

    # Join the distinct store/dept pairs to the training set.
    # Why left join? When would training data not be available?
    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    
    # Add numeric column for the year and a categorical column for week # to the training set
    train_split = preprocess(train_split)
    # Get design matrices for training y and X.
    # y is just the target variable, Weekly_Sales.
    # X has pivoted weeks, where individual weeks are separate 0/1 columns.
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + Wk', 
                        data = train_split, 
                        return_type='dataframe')
    # Get dictionary where keys are (Store, Dept) tuples, and values are the
    # \"Weekly_Sales + Store + Dept + Yr + Wk\" design matrices corresponding to each key.
    # The design matrices include an Intercept column with value 1.
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    
    # Now join the distinct store/dept pairs to the test set.
    # Same question: why left join? When would training data not be available?
    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    # Add numeric column for the year and a categorical column for week # to the test set
    test_split = preprocess(test_split)
    # Get design matrices for text y and X.
    # y is the Year, and the design matrix is \"Store + Dept + Yr + Wk\".
    # Note that test sets don't have the Weekly_Sales target variable.
    # Why save Year as y?
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk', 
                        data = test_split, 
                        return_type='dataframe')
    # Re-add Date column to the design matrix X
    X['Date'] = test_split['Date']
    # Get dictionary where keys are (Store, Dept) tuples, and values are the
    # \"Yr  + Wk + Date\" design matrices corresponding to each key.
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    # Get the training (store, dept) tuples.
    # SHOULD be the same keys as in test, given the left joins above.
    keys = list(train_split)

    # Loop over (store, dept) tuples
    for key in keys:
        # Get training and test design matrices corresponding to (store, dept)
        X_train = train_split[key]
        X_test = test_split[key]
    
        # Target variable for (store, dept)
        Y = X_train['Weekly_Sales']
        # Drop ID and target to get just a table of predictors
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)
        
        # Identify columns that are all zero in training predictors, and drop them
        # from both training and test X.
        # This should drop weeks that are not represented in the training data.
        # How does this affect test X? Are there cases where all test weeks would be dropped?
        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        
        # Identify X training columns that are highly collinear with the columns to the left.
        # Note that this doesn't check the Intercept column.
        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)
                
        # Drop those collinear columns from both training and test X.
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
        print(X_train)
        # Fit a regular ordinary least squares model on training Weekly_Sales.
        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)
        
        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)
        
        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
        
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    # Save the output to CSV
    file_path = f'Data/fold_{j+1}/mypred.csv'
    print(f'fold_{j+1} processed')
    test_pred.to_csv(file_path, index=False)

## Evaluate Predictions

In [ ]:
wae = wmae()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

Linear errors

	2049.347
	1467.113
	1446.882
	1595.628
	2334.678
	1675.221
	1720.828
	1427.286
	1443.787
	1444.677
1660.545

## Edit original OLS: group stores by year and add SVD/PCA

In [19]:
# Try on a single fold: j = 2
j = 2

# Components to return from SVD. This is from the example in Campuswire post #364:
# https://campuswire.com/c/G06C55090/feed/364
n_components = 8
temp_seed = 4031
    
# Get a pair of training and test sets
train = train_datasets[j]
test = test_datasets[j]

test_pred = pd.DataFrame()

# Identify the distinct store/dept pairs shared by the training and test set.
# Will only process these.

train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])

# Join the distinct store/dept pairs to the training set.
# Why left join? When would training data not be available?
train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')

In [20]:
# Get unique stores and dates. This is to help fill in zeroes for any departments
# that are missing from certain stores and dates.

train_store_list = train_split["Store"].unique().tolist()
train_date_list = train_split["Date"].unique().tolist()

In [30]:
# Prep train data for SVD/PCA.
# For each department, construct a dataframe consisting of:
# rows = dates; columns = store numbers; and values = weekly sales.


# Get columns needed for SVD
train_store_dept_date_sales = train_split[["Store", "Dept", "Date", "Weekly_Sales"]]

# Pivot dataframe so that sales dates are rows and stores are columns, with values = Weekly_Sales.
# Fill in missing values with zeroes.
train_pivot = train_store_dept_date_sales.pivot(index=["Dept", "Date"], columns="Store", values="Weekly_Sales").reset_index().fillna(0)

# Fill in any missing dates for any stores and departments.
train_pivot["Date"] = pd.Categorical(train_pivot["Date"], categories=train_pivot["Date"].unique())
train_pivot = train_pivot.groupby(["Dept", "Date"], as_index=False).first()

In [31]:
train_pivot

Store,Dept,Date,1,2,3,4,5,6,7,8,...,36,37,38,39,40,41,42,43,44,45
0,1,2010-02-05,24924.50,35034.06,6453.58,38724.42,9323.89,25619.00,8970.97,16181.89,...,2144.48,11283.23,6732.38,21244.50,18116.85,16971.05,10425.77,6476.76,6871.20,18628.11
1,1,2010-02-12,46039.49,60483.70,12748.72,69872.44,16861.10,43749.81,14026.65,34262.09,...,4091.72,16184.33,9132.55,39584.16,26138.18,30204.01,15725.68,18597.64,12315.65,22416.94
2,1,2010-02-19,41595.55,58221.52,8918.31,49937.09,11417.67,34750.82,12477.79,22319.25,...,3101.57,10722.08,8045.28,23025.91,23172.75,20694.24,13300.99,9939.45,7751.11,28756.53
3,1,2010-02-26,19403.54,25962.32,4992.00,30107.54,7168.41,19896.08,8602.73,11722.71,...,1451.39,9256.03,5951.54,14011.39,14728.82,12816.16,9303.34,6460.56,6014.71,14656.08
4,1,2010-03-05,21827.90,27372.05,5172.73,31580.69,8344.13,22839.36,9541.12,12979.74,...,1510.25,9766.32,6485.19,14875.08,18494.41,15154.51,10244.26,6939.08,6120.60,16608.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,99,2011-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5763,99,2011-06-03,0.00,-20.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5764,99,2011-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5765,99,2011-06-17,50.00,50.00,0.00,150.00,0.00,200.00,0.00,0.00,...,0.00,0.00,0.00,0.00,50.00,100.00,0.00,0.00,0.00,0.00


In [33]:
# Convert dates from categorical back to datetime
train_pivot["Date"] = pd.to_datetime(train_pivot["Date"], infer_datetime_format = True, utc = True).astype('datetime64[ns]')

C:\Users\tuohy\AppData\Local\Temp\ipykernel_13012\1936529210.py:2: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  train_pivot["Date"] = pd.to_datetime(train_pivot["Date"], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


In [34]:
# For the filled-in dates, replace missing sales figures with zeroes
train_pivot = train_pivot.fillna(0)

In [41]:
# Set all column names to string
train_pivot.columns = train_pivot.columns.astype(str)

In [42]:
# Group by Dept. Create dict where key = Dept and value = dataframe of Store/Date/Weekly_Sales.
train_split = dict(tuple(train_pivot.groupby(["Dept"])))
train_split

{1: Store  Dept       Date         1         2         3         4         5  \
 0         1 2010-02-05  24924.50  35034.06   6453.58  38724.42   9323.89   
 1         1 2010-02-12  46039.49  60483.70  12748.72  69872.44  16861.10   
 2         1 2010-02-19  41595.55  58221.52   8918.31  49937.09  11417.67   
 3         1 2010-02-26  19403.54  25962.32   4992.00  30107.54   7168.41   
 4         1 2010-03-05  21827.90  27372.05   5172.73  31580.69   8344.13   
 ..      ...        ...       ...       ...       ...       ...       ...   
 68        1 2011-05-27  15741.60  21334.34   5194.21  25035.61   7901.75   
 69        1 2011-06-03  16434.15  22174.12   4906.27  25401.86   8175.25   
 70        1 2011-06-10  15883.52  21554.22   4790.03  24726.22   8170.13   
 71        1 2011-06-17  14978.09  22431.56   5738.30  27152.27   8157.72   
 72        1 2011-06-24  15682.81  21426.72   4484.47  26859.67   8392.34   
 
 Store         6         7         8  ...       36        37       38  

In [43]:
# Split sales dates out from remaining data
train_sales_dates = {dept:df.Date for dept, df in train_split.items()}
# Get just the sales figures for each dept
train_X = {dept:df.drop(columns=["Date"]) for dept, df in train_split.items()}

#train_X[1]
#{dept:df.shape for dept, df in train_X.items()}

# Perform SVD and choose the n_components most influential components.
svd = TruncatedSVD(n_components=n_components, random_state=temp_seed)
#svd.fit_transform(train_X[1])
# Collect dict of reduced-dimension training data, one entry per department.
svd_dict = {dept:svd.fit_transform(df) for dept, df in train_X.items()}
svd_dict[1]

array([[ 1.48807268e+05, -9.99353518e+03, -3.18608992e+01,
        -4.37803668e+03, -2.67268942e+03, -3.67391552e+03,
        -3.71424133e+03, -1.91466774e+03],
       [ 2.41045082e+05, -3.02587043e+04,  1.69425700e+04,
         1.30125058e+04,  7.44887886e+03, -1.43381725e+04,
        -6.22496807e+03,  4.44900954e+03],
       [ 1.84280398e+05, -2.67636306e+04,  1.77219628e+04,
        -1.38735259e+04, -7.91462077e+03, -8.82273110e+03,
        -8.56183003e+03, -1.38089211e+02],
       [ 1.12611339e+05, -1.28745480e+04,  6.13199126e+01,
        -4.78428038e+03, -7.68655358e+03, -1.06198671e+02,
         2.69389257e+03,  1.27911028e+03],
       [ 1.26443571e+05, -9.42570471e+03, -1.85355589e+03,
        -6.87753006e+03, -9.43867656e+03, -6.61670039e+02,
         9.00023622e+02,  1.90892529e+03],
       [ 1.35863547e+05, -2.56713771e+03, -5.22163213e+03,
        -8.46320284e+03, -5.94078955e+03, -5.81386097e+02,
        -7.32659402e+02, -2.34759161e+03],
       [ 1.45204659e+05,  3.420466